In [59]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

In [60]:
URL = "https://www.amazon.pl/s?k=ekspres+ci%C5%9Bnieniowy&__mk_pl_PL=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=249AWWJCXNNBX&sprefix=ekspres+ci%C5%9Bnieniowy%2Caps%2C85&ref=nb_sb_noss_1"

In [61]:
# Headers for request
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.42'

HEADERS = ({'User-Agent':user_agent, 'Accept-Language': 'pl-PL, pl; q=0.5'})

In [62]:
# HTTP Request
webpage = requests.get(URL, headers = HEADERS)

#<Response [200]> == Success
webpage

<Response [200]>

In [63]:
type(webpage.content)

bytes

In [64]:
# soup object containing all data
soup = BeautifulSoup(webpage.content, 'html.parser')

In [65]:
# Fetch links as list of tag objects
links = soup.find_all("a", attrs={'class': 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})
max_site = int(soup.find("span", attrs={"class": "s-pagination-item s-pagination-disabled"}).text)


In [66]:
links_list = []

for link in links:
    adress = link.get('href')
    links_list.append(adress)

In [67]:
def get_title(x):
    try:
        title = x.find("span", attrs={"id": "productTitle"}).text.strip()
        return title

    except AttributeError:
        return ""

def get_brand(x):
    try:
        brand = x.find("td", attrs={"class": "a-span9"}).text.strip()
        return brand

    except AttributeError:
        return ""

def get_price(x):
    try:
        price = x.find("span", attrs={"class": "a-offscreen"}).text.strip().replace("zł","").replace(',','.')
        return price

    except AttributeError:
        return ""

In [68]:
data = {'title': [],"brand":[], "price": []}

for link in links_list:
    # HTTP Requests
    product_link = "https://amazon.pl" + link
    new_webpage = requests.get(product_link, headers = HEADERS)
    
    # soup object containing all data
    new_soup = BeautifulSoup(new_webpage.content, 'html.parser')

    
    data['title'].append(get_title(new_soup))
    data['brand'].append(get_brand(new_soup))
    data['price'].append(get_price(new_soup))



In [69]:
# Pages 2 - MAX

start = 2
stop = max_site + 1
progress = start
while progress < stop:
    URL = "https://www.amazon.pl/s?k=ekspres+ci%C5%9Bnieniowy&page={}&__mk_pl_PL=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=249AWWJCXNNBX&qid=1668784032&sprefix=ekspres+ci%C5%9Bnieniowy%2Caps%2C85&ref=sr_pg_{}".format(progress, progress - 1)
    print(URL)
    progress += 1

    # HTTP Request
    webpage = requests.get(URL, headers = HEADERS)

    # soup object containing all data
    soup = BeautifulSoup(webpage.content, 'html.parser')

    # Fetch links as list of tag objects
    links = soup.find_all("a", attrs={'class': 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    links_list = []

    for link in links:
        adress = link.get('href')
        links_list.append(adress)

    for link in links_list:
        # HTTP Requests
        product_link = "https://amazon.pl" + link
        new_webpage = requests.get(product_link, headers = HEADERS)
        
        # soup object containing all data
        new_soup = BeautifulSoup(new_webpage.content, 'html.parser')

        
        data['title'].append(get_title(new_soup))
        data['brand'].append(get_brand(new_soup))
        data['price'].append(get_price(new_soup))



https://www.amazon.pl/s?k=ekspres+ci%C5%9Bnieniowy&page=2&__mk_pl_PL=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=249AWWJCXNNBX&qid=1668784032&sprefix=ekspres+ci%C5%9Bnieniowy%2Caps%2C85&ref=sr_pg_1
https://www.amazon.pl/s?k=ekspres+ci%C5%9Bnieniowy&page=3&__mk_pl_PL=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=249AWWJCXNNBX&qid=1668784032&sprefix=ekspres+ci%C5%9Bnieniowy%2Caps%2C85&ref=sr_pg_2
https://www.amazon.pl/s?k=ekspres+ci%C5%9Bnieniowy&page=4&__mk_pl_PL=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=249AWWJCXNNBX&qid=1668784032&sprefix=ekspres+ci%C5%9Bnieniowy%2Caps%2C85&ref=sr_pg_3
https://www.amazon.pl/s?k=ekspres+ci%C5%9Bnieniowy&page=5&__mk_pl_PL=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=249AWWJCXNNBX&qid=1668784032&sprefix=ekspres+ci%C5%9Bnieniowy%2Caps%2C85&ref=sr_pg_4
https://www.amazon.pl/s?k=ekspres+ci%C5%9Bnieniowy&page=6&__mk_pl_PL=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=249AWWJCXNNBX&qid=1668784032&sprefix=ekspres+ci%C5%9Bnieniowy%2Caps%2C85&ref=sr_pg_5
https://www.amazon.pl/s?k=ekspres+ci%C5%9Bnie

In [82]:
amazon_df = pd.DataFrame.from_dict(data)
amazon_df['title'].replace('', np.nan, inplace=True)
amazon_df = amazon_df.dropna(subset=['title'])
amazon_df.to_csv('amazon_data.csv', header=True, index=False, sep=';')

In [71]:
amazon_df.head()

,title,brand,price
0,"Russell Hobbs ekspres do kawy, ciśnieniowy, 15...",Russell Hobbs,1 179.00
1,"Russell Hobbs ekspres do kawy, ciśnieniowy, 15...",Russell Hobbs,999.99
2,Saeco 229452100 oryginalny zawór ciśnieniowy z...,,86.39
3,"Ufesa CE 7238 Cream - Ekspres do kawy, do przy...",UFESA,499.00
4,De'Longhi Stilosa Advanced EC235.BK ekspres do...,De'Longhi,405.69
